# Address resolution protocol (ARP)

In this experiment, we will examine how ARP is used in IPv4 networks -

-   to resolve IPv4 addresses of neighbors into link layer MAC addresses
-   and to keep track of the neighbor reachability.

It should take about 60 minutes to run this experiment.

In this notebook you will:

-   Reserve resources for this experiment
-   Configure your reserved resources
-   Access your reserved resources over SSH
-   Delete your resources when you are finished

To *execute* the experiment, you will follow the instructions at: https://witestlab.poly.edu/blog/address-resolution-protocol-arp/

### Configure environment

In [ ]:
import openstack, chi, chi.ssh, chi.network, chi.server, os

In this section, we configure the Chameleon Python client.

For this experiment, we’re going to use the KVM@TACC site, which we indicate below.

We also need to specify the name of the Chameleon “project” that this experiment is part of. The project name will have the format “CHI-XXXXXX”, where the last part is a 6-digit number, and you can find it on your [user dashboard](https://chameleoncloud.org/user/dashboard/).

In the cell below, replace the project ID with your own project ID, then run the cell.

In [ ]:
chi.use_site("KVM@TACC")
PROJECT_NAME = "CHI-XXXXXX"
chi.set("project_name", PROJECT_NAME)

# configure openstacksdk for actions unsupported by python-chi
os_conn = chi.clients.connection()


### Define configuration for this experiment (two hosts and router\` in line topology)

In [ ]:
username = os.getenv('USER')

node_conf = [
 {'name': "romeo",   'flavor': 'm1.small', 'image': 'CC-Ubuntu22.04', 'packages': []}, 
 {'name': "juliet",  'flavor': 'm1.small', 'image': 'CC-Ubuntu22.04', 'packages': []}, 
 {'name': "hamlet",  'flavor': 'm1.small', 'image': 'CC-Ubuntu22.04', 'packages': []},
]
net_conf = [
 {"name": "net0", "subnet": "10.0.0.0/24", "nodes": [{"name": "romeo",   "addr": "10.0.0.100"}, {"name": "juliet", "addr": "10.0.0.101"}, {"name": "hamlet", "addr": "10.0.0.102"}]}
]
route_conf = []

### Configure resources

Now, we will prepare the VMs and network links that our experiment requires.

First, we will prepare a “public” network that we will use for SSH access to our VMs -

In [ ]:
public_net = os_conn.network.create_network(name="public_net_" + username)
public_net_id = public_net.get("id")
public_subnet = os_conn.network.create_subnet(
    name="public_subnet_" + username,
    network_id=public_net.get("id"),
    ip_version='4',
    cidr="192.168.10.0/24",
    gateway_ip="192.168.10.1",
    is_dhcp_enabled = True
)

Next, we will prepare the “experiment” networks -

In [ ]:
nets = []
net_ids = []
subnets = []
for n in net_conf:
    exp_net = os_conn.network.create_network(name="exp_" + n['name']  + '_' + username)
    exp_net_id = exp_net.get("id")
    os_conn.network.update_network(exp_net, is_port_security_enabled=False)
    exp_subnet = os_conn.network.create_subnet(
        name="exp_subnet_" + n['name']  + '_' + username,
        network_id=exp_net.get("id"),
        ip_version='4',
        cidr=n['subnet'],
        gateway_ip=None,
        is_dhcp_enabled = False
    )
    nets.append(exp_net)
    net_ids.append(exp_net_id)
    subnets.append(exp_subnet)

Now we create the VMs -

In [ ]:
servers = []
server_ids = []
for i, n in enumerate(node_conf, start=10):
    image_uuid = os_conn.image.find_image(n['image']).id
    flavor_uuid = os_conn.compute.find_flavor(n['flavor']).id
    # find out details of exp interface(s)
    nics = [{'net-id': chi.network.get_network_id( "exp_" + net['name']  + '_' + username ), 'v4-fixed-ip': node['addr']} for net in net_conf for node in net['nodes'] if node['name']==n['name']]
    # also include a public network interface
    nics.insert(0, {"net-id": public_net_id, "v4-fixed-ip":"192.168.10." + str(i)})
    server = chi.server.create_server(
        server_name=n['name'] + "_" + username,
        image_id=image_uuid,
        flavor_id=flavor_uuid,
        nics=nics
    )
    servers.append(server)
    server_ids.append(chi.server.get_server(n['name'] + "_" + username).id)

We wait for all servers to come up before we proceed -

In [ ]:
for server_id in server_ids:
    chi.server.wait_for_active(server_id)

Next, we will set up SSH access to the VMs.

First, we will make sure the “public” network is connected to the Internet. Then, we will configure it to permit SSH access on port 22 for each port connected to this network.

In [ ]:
# connect them to the Internet on the "public" network (e.g. for software installation)
router = chi.network.create_router('inet_router_' + username, gw_network_name='public')
chi.network.add_subnet_to_router(router.get("id"), public_subnet.get("id"))

In [ ]:
# prepare SSH access on the servers
# WARNING: this relies on undocumented behavior of associate_floating_ip 
# that it associates the IP with the first port on the server
server_ips = []
for server_id in server_ids:
    ip = chi.server.associate_floating_ip(server_id)
    server_ips.append(ip)

Note: The following cells assumes that a security group named “Allow SSH” already exists in your project, and is configured to allow SSH access on port 22. If you have done the “Hello, Chameleon” experiment then you already have this security group.

In [ ]:
security_group_id = os_conn.get_security_group("Allow SSH").id
for port in chi.network.list_ports(): 
    if port['port_security_enabled'] and port['network_id']==public_net.get("id"):
        os_conn.network.update_port(port['id'], security_groups=[security_group_id])

We also copy our account keys to all of the servers -

In [ ]:
for ip in server_ips:
    chi.server.wait_for_tcp(ip, port=22)

In [ ]:
server_remotes = [chi.ssh.Remote(ip) for ip in server_ips]

In [ ]:
nova=chi.clients.nova()
# iterate over all keypairs in this account
for kp in nova.keypairs.list(): 
    public_key = nova.keypairs.get(kp.name).public_key 
    for remote in server_remotes:
        remote.run(f"echo {public_key} >> ~/.ssh/authorized_keys")     

Finally, we need to configure our resources, including software package installation and network configuration.

In [ ]:
# configure an IP address on each port that is supposed to have one
for port in chi.network.list_ports():
    if port['network_id'] in net_ids:
        i = server_ids.index(port['device_id'])
        j = net_ids.index(port['network_id'])
        port_conf =  [item for item in net_conf[j]['nodes'] if item['name'] == node_conf[i]['name'] ][0]
        if port_conf['addr']:
            server_remotes[i].run( "sudo ip addr flush dev $(ip --br link | grep '" + port['mac_address'] + "' | awk '{print $1}')" )
            cmd_prefix = "sudo ip addr add " + port_conf['addr'] + "/" + net_conf[j]['subnet'].split("/")[1] 
            server_remotes[i].run( cmd_prefix + " dev $(ip --br link | grep '" + port['mac_address'] + "' | awk '{print $1}')" )
        else:
            server_remotes[i].run( "sudo ip addr flush dev $(ip --br link | grep '" + port['mac_address'] + "' | awk '{print $1}')" )

In [ ]:
for i, n in enumerate(node_conf):
    remote = server_remotes[i]
    # enable forwarding
    remote.run(f"sudo sysctl -w net.ipv4.ip_forward=1") 
    remote.run(f"sudo ufw disable") 
    # configure static routes
    for r in route_conf: 
        if n['name'] in r['nodes']:
            remote.run(f"sudo ip route add " + r['addr'] + " via " + r['gw']) 

In [ ]:
for i, n in enumerate(node_conf):
    # install packages
    if len(n['packages']):
            remote.run(f"sudo apt update; sudo apt -y install " + " ".join(n['packages'])) 

In [ ]:
# prepare a "hosts" file that has names and addresses of every node
hosts_txt = [ "%s\t%s" % ( n['addr'], n['name'] ) for net in net_conf  for n in net['nodes'] if type(n) is dict and n['addr']]
for remote in server_remotes:
    for h in hosts_txt:
        remote.run("echo %s | sudo tee -a /etc/hosts > /dev/null" % h)

In [ ]:
# turn segment offload off
for port in chi.network.list_ports():
    if port['network_id'] in net_ids:
        i = server_ids.index(port['device_id'])
        j = net_ids.index(port['network_id'])
        for offload in ["gro", "gso", "tso"]:
            server_remotes[i].run( "sudo ethtool -K $(ip --br link | grep '" + port['mac_address'] + "' | awk '{print $1}') " + offload + " off" )

### Draw the network topology

The following cells will draw the network topology, for your reference.

In [ ]:
!pip install networkx

In [ ]:
nodes = [ (n['name'], {'color': 'pink'}) for n in net_conf ] + [(n['name'], {'color': 'lightblue'}) for n in node_conf ]
edges = [(net['name'], node['name'], 
          {'label': node['addr'] + '/' + net['subnet'].split("/")[1] }) if node['addr'] else (net['name'], node['name']) for net in net_conf for node in net['nodes'] ]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);

### Log in to resources

At this point, we should be able to log in to our resources over SSH! Run the following cell, and observe the output - you will see an SSH command for each of the nodes in your topology.

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n['name'], 'SSH command': "ssh cc@" + server_ips[i] } for i, n in enumerate(node_conf)]
pd.DataFrame(slice_info).set_index('Name')

Now, you can open an SSH session on any of the nodes as follows:

-   In Jupyter, from the menu bar, use File \> New \> Terminal to open a new terminal.
-   Copy an SSH command from the output above, and paste it into the terminal.
-   You can repeat this process (open several terminals) to start a session on each node. Each terminal session will have a tab in the Jupyter environment, so that you can easily switch between them.

Alternatively, you can use your local terminal to log on to each node, if you prefer. (On your local terminal, you may need to also specify your key path as part of the SSH command, using the `-i` argument followed by the path to your private key.)

### Delete resources

When you finish your experiment, you should delete your resources! The following cells deletes all the resources in this experiment, freeing them for other experimenters.

In [ ]:
# delete the nodes
for server_id in server_ids:
    chi.server.delete_server(server_id)

In [ ]:
# release the floating IP addresses used for SSH
for server_ip in server_ips:
    ip_details = chi.network.get_floating_ip(server_ip)
    chi.neutron().delete_floatingip(ip_details["id"])

In [ ]:
# delete the router used for public Internet access
chi.network.remove_subnet_from_router(router.get("id"), public_subnet.get("id"))
chi.network.delete_router(router.get("id"))

In [ ]:
# delete the public network
chi.network.delete_subnet(public_subnet.get('id'))
chi.network.delete_network(public_net.get("id"))

In [ ]:
# delete the experiment networks
for subnet, net in zip(subnets, nets):
    chi.network.delete_subnet(subnet.get('id'))
    chi.network.delete_network(net.get('id'))